TokaMax Example: Pulse design example in ITER {#doc_tMax_ITER_ex1}
==========
In this example we show how to use TokaMax for a simple pulse in ITER.
Note: this example requires torax to be installed.

In [1]:
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import torax

plt.rcParams['figure.figsize']=(6,6)
plt.rcParams['font.weight']='bold'
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['lines.linewidth']=2
plt.rcParams['lines.markeredgewidth']=2
%matplotlib inline
%config InlineBackend.figure_format = "retina"

14:30:23 CRITICAL Could not import 'imas_core': No module named 'imas_core'. Some functionality is not available. @imas_interface.py:34


## Load TokaMaker library

In [2]:
tokamaker_python_path = os.getenv('OFT_ROOTPATH')
if tokamaker_python_path is not None:
    sys.path.append(os.path.join(tokamaker_python_path,'python'))
from OpenFUSIONToolkit.TokaMax import TokaMax

## Load EQDSK files

In [3]:
# Set timesteps
rampup_times = np.linspace(5.0, 80.0, 10)
flattop_times = np.linspace(100.0, 450.0, 10)
rampdown_times = np.linspace(500.0, 600.0, 10)
times = np.r_[rampup_times, flattop_times, rampdown_times]

# Load gEQDSK
g_arr_rampup = [f'ITER-eqdsk/rampup_i={i}.eqdsk' for i in range(10)]
g_arr_flattop = ['ITER-eqdsk/Hmode.eqdsk'] * 10
g_arr_rampdown = g_arr_rampup[::-1]
g_arr = np.r_[g_arr_rampup, g_arr_flattop, g_arr_rampdown]


## Create TokaMax solver object

In [4]:
mysim = TokaMax(0, 600, times, g_arr, last_surface_factor=0.9)
mysim.initialize_gs('ITER_mesh.h5', vsc='VS')

coils = ['CS3U', 'CS2U', 'CS1U', 'CS1L', 'CS2L', 'CS3L', 'PF1', 'PF2', 'PF3', 'PF4', 'PF5', 'PF6']
coil_targets = {coil_name: 0.0 for coil_name in coils}
mysim.set_coil_reg(coil_targets, strict_limit=1.0E8)


#----------------------------------------------
Open FUSION Toolkit Initialized
Development branch:   Add-TokaMax
Revision id:          79a729e
Parallelization Info:
  Not compiled with MPI
  # of OpenMP threads =    2
Fortran input file    = /var/folders/qz/wcp9q1mj0c5fsyjtbt99bvhh0000gn/T/oft_24049/oftpyin
XML input file        = none
Integer Precisions    =    4   8
Float Precisions      =    4   8  16
Complex Precisions    =    4   8
LA backend            = native
#----------------------------------------------


**** Loading OFT surface mesh

**** Generating surface grid level  1
  Generating boundary domain linkage
  Mesh statistics:
    Area         =  2.859E+02
    # of points  =    4757
    # of edges   =   14156
    # of cells   =    9400
    # of boundary points =     112
    # of boundary edges  =     112
    # of boundary cells  =     112
  Resolution statistics:
    hmin =  9.924E-03
    hrms =  2.826E-01
    hmax =  8.466E-01
  Surface grounded at vertex     870


**** C

## Set Actuators and Boundary Conditions

In [5]:
# Set current
ip = {0: 3.0E6, 5: 3.0E6, 80: 15.0E6, 500: 15.0E6, 590: 4.0E6, 600: 4.0E6}

# Set heating
powers = {0: 0, 24: 0, 25: 10.0E6, 79: 10.0E6, 80: 52.0E6, 124: 52.0E6, 125: 40.0E6, 500: 40.0E6, 524: 40.0E6, 525: 35.0E6, 549: 35.0E6, 550: 30.0E6}
nbi_powers = {k: 0.5 * v for k, v in powers.items()}
eccd_powers = {k: 0.5 * v for k, v in powers.items()}

# Set pedestals
T_i_ped = {0: 0.146, 80: 0.146, 85: 3.69, 500: 3.69, 505: 0.146}
T_e_ped = {0: 0.220, 80: 0.220, 85: 3.69, 500: 3.69, 505: 0.220}
n_e_ped = {0: 1.821E19, 79: 1.821E19, 80: 7.482E19}

# Set boundary conditions
ne_right_bc = {0: 0.157E20, 79: 0.157E20, 80: 0.414E20}
Te_right_bc = 0.01
Ti_right_bc = 0.01

mysim.set_Ip(ip)
mysim.set_Zeff(1.8)
mysim.set_heating(nbi=nbi_powers, nbi_loc=0.25, eccd=eccd_powers, eccd_loc=0.35)
mysim.set_right_bc(Te_right_bc=Te_right_bc, Ti_right_bc=Ti_right_bc, ne_right_bc=ne_right_bc)
mysim.set_pedestal(T_i_ped=T_i_ped, T_e_ped=T_e_ped, n_e_ped=n_e_ped)
mysim.set_nbar({0: 0.326E20, 80: .905E20})

## Run Sim

In [ ]:
mysim.fly(max_step=5, remove_tmp=True, graph=False)

INFO:2025-11-13 14:30:25,445:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/opt/homebrew/anaconda3/bin/../lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)
2025-11-13 14:30:25,445 [    INFO] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/opt/homebrew/anaconda3/bin/../lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)
2025-11-13 14:30:25,477 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbi

Starting non-linear GS solver
     1  2.6145E+00  2.6462E-03  1.1622E-01  6.7125E+00  4.7131E-01  2.7029E-10
     2  2.6717E+00  2.7103E-03  7.0345E-02  6.6991E+00  4.8701E-01  1.3666E-10
     3  2.6868E+00  2.7274E-03  3.2105E-02  6.6946E+00  4.9426E-01  1.5784E-10
     4  2.6911E+00  2.7325E-03  1.2878E-02  6.6931E+00  4.9753E-01  1.7943E-10
     5  2.6924E+00  2.7340E-03  4.9642E-03  6.6925E+00  4.9896E-01  1.5710E-10
     6  2.6928E+00  2.7345E-03  1.8931E-03  6.6923E+00  4.9958E-01  1.5698E-10
     7  2.6929E+00  2.7346E-03  7.2069E-04  6.6923E+00  4.9984E-01  1.5693E-10
     8  2.6929E+00  2.7347E-03  2.7701E-04  6.6922E+00  4.9995E-01  1.5691E-10
     9  2.6929E+00  2.7347E-03  1.0786E-04  6.6922E+00  5.0000E-01  1.5690E-10
    10  2.6929E+00  2.7347E-03  4.2503E-05  6.6922E+00  5.0002E-01  1.5690E-10
    11  2.6929E+00  2.7347E-03  1.6900E-05  6.6922E+00  5.0002E-01  1.5690E-10
    12  2.6929E+00  2.7347E-03  6.7679E-06  6.6922E+00  5.0003E-01  1.5690E-10
    13  2.6929E+00  2.

Saving gEQDSK: tmp/001.028.eqdsk
 Using COCOS=2...
Starting non-linear GS solver
     1  2.1356E+00  1.9086E-02  1.6379E-01  6.7612E+00  4.7154E-01 -1.4011E-03
     2  2.1889E+00  1.9545E-02  1.0395E-01  6.7620E+00  4.8731E-01  2.9925E-03
     3  2.2027E+00  1.9664E-02  4.7381E-02  6.7625E+00  4.9457E-01  4.4102E-03
     4  2.2067E+00  1.9699E-02  1.8636E-02  6.7627E+00  4.9785E-01  4.8751E-03
     5  2.2079E+00  1.9709E-02  7.0959E-03  6.7627E+00  4.9928E-01  5.0350E-03
     6  2.2083E+00  1.9712E-02  2.6422E-03  6.7628E+00  4.9990E-01  5.0890E-03
     7  2.2084E+00  1.9713E-02  9.8919E-04  6.7628E+00  5.0016E-01  5.1069E-03
     8  2.2085E+00  1.9713E-02  3.7517E-04  6.7628E+00  5.0027E-01  5.1128E-03
     9  2.2085E+00  1.9713E-02  1.4399E-04  6.7628E+00  5.0032E-01  5.1148E-03
    10  2.2085E+00  1.9713E-02  5.5973E-05  6.7628E+00  5.0034E-01  5.1154E-03
    11  2.2085E+00  1.9713E-02  2.2010E-05  6.7628E+00  5.0035E-01  5.1156E-03
    12  2.2085E+00  1.9713E-02  8.7398E-06  6.7628

/Users/johnl/Desktop/OpenFUSIONToolkit-SOL-Currents/install_release/python/OpenFUSIONToolkit/TokaMax/_core.py:941: RuntimeWarning: divide by zero encountered in scalar divide
  err = np.abs(cflux - cflux_prev) / cflux_prev
2025-11-13 14:30:56,713 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:30:56,728 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:30:56,7

    13  2.2085E+00  1.9713E-02  3.4979E-06  6.7628E+00  5.0035E-01  5.1156E-03
    14  2.2085E+00  1.9713E-02  1.4086E-06  6.7628E+00  5.0035E-01  5.1157E-03
    15  2.2085E+00  1.9713E-02  5.6983E-07  6.7628E+00  5.0035E-01  5.1157E-03
 Timing:  0.11643599718809128     
   Source:     3.7432007491588593E-002
   Solve:      3.7584003061056137E-002
   Boundary:   3.5289935767650604E-003
   Other:      3.7890993058681488E-002
Equilibrium Statistics:
  Topology                =   Limited
  Toroidal Current [A]    =    4.0000E+06
  Current Centroid [m]    =    6.625  0.500
  Magnetic Axis [m]       =    6.763  0.500
  Elongation              =    1.399 (U:  1.398, L:  1.399)
  Triangularity           =   -0.011 (U: -0.016, L: -0.005)
  Plasma Volume [m^3]     =   567.538
  q_0, q_95               =    2.561  4.497
  Plasma Pressure [Pa]    =   Axis:  6.7664E+04, Peak:  6.7664E+04
  Stored Energy [J]       =    2.0558E+07
  <Beta_pol> [%]          =   43.0283
  <Beta_tor> [%]          =    

...

Starting non-linear GS solver
     1  2.6044E+00  2.7632E-03  1.1613E-01  6.7130E+00  4.7131E-01 -1.4624E-02
     2  2.6614E+00  2.8301E-03  7.0376E-02  6.6998E+00  4.8702E-01 -1.3222E-02
     3  2.6764E+00  2.8480E-03  3.2113E-02  6.6953E+00  4.9426E-01 -1.2814E-02
     4  2.6807E+00  2.8533E-03  1.2880E-02  6.6938E+00  4.9753E-01 -1.2689E-02
     5  2.6820E+00  2.8549E-03  4.9682E-03  6.6932E+00  4.9896E-01 -1.2652E-02
     6  2.6824E+00  2.8554E-03  1.8979E-03  6.6930E+00  4.9958E-01 -1.2641E-02
     7  2.6825E+00  2.8555E-03  7.2502E-04  6.6930E+00  4.9984E-01 -1.2638E-02
     8  2.6825E+00  2.8556E-03  2.7960E-04  6.6930E+00  4.9995E-01 -1.2637E-02
     9  2.6825E+00  2.8556E-03  1.0900E-04  6.6930E+00  5.0000E-01 -1.2637E-02
    10  2.6825E+00  2.8556E-03  4.2928E-05  6.6929E+00  5.0002E-01 -1.2637E-02
    11  2.6825E+00  2.8556E-03  1.7058E-05  6.6929E+00  5.0002E-01 -1.2637E-02
    12  2.6825E+00  2.8556E-03  6.8242E-06  6.6929E+00  5.0003E-01 -1.2637E-02
    13  2.6825E+00  2.

    13  8.9413E+00  8.1024E-03  1.1237E-06  6.3665E+00  5.0086E-01  1.9295E-02
    14  8.9413E+00  8.1024E-03  4.7187E-07  6.3665E+00  5.0086E-01  1.9295E-02
 Timing:  0.10938899964094162     
   Source:     3.5299997776746750E-002
   Solve:      3.5235013812780380E-002
   Boundary:   4.0210075676441193E-003
   Other:      3.4832980483770370E-002
Equilibrium Statistics:
  Topology                =   Limited
  Toroidal Current [A]    =    1.5000E+07
  Current Centroid [m]    =    6.156  0.500
  Magnetic Axis [m]       =    6.366  0.501
  Elongation              =    1.528 (U:  1.528, L:  1.527)
  Triangularity           =   -0.016 (U: -0.023, L: -0.009)
  Plasma Volume [m^3]     =   774.333
  q_0, q_95               =    1.226  2.069
  Plasma Pressure [Pa]    =   Axis:  9.3778E+04, Peak:  9.3778E+04
  Stored Energy [J]       =    3.8786E+07
  <Beta_pol> [%]          =    6.2638
  <Beta_tor> [%]          =    0.3029
  <Beta_n>   [%]          =    0.2160
  Diamagnetic flux [Wb]   =    2.1

2025-11-13 14:31:52,969 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:31:52,985 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:31:53,002 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Pl

Saving gEQDSK: tmp/004.029.eqdsk
 Using COCOS=2...


2025-11-13 14:31:53,096 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:31:53,112 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Please verify carefully.
               Future work will be done to correctly handle EQDSK inputs provided with a specific COCOS value.
2025-11-13 14:31:53,128 [ WARNING] 
               The TORAX EQDSK converter has only been tested against CHEASE-generated EQDSK which is COCOS=2.
               The converter is not guaranteed to work as expected with arbitrary EQDSK input. Pl

## Graph the results

In [ ]:
fname = 'res.json'

res = None
with open(fname, 'r') as f:
    res = json.loads(f.read())

fig, ax = plt.subplots(3, 2) #, figsize=(20,10))

ax[0,0].set_title('Heating (MW)')
ax[0,0].plot(res['P_alpha_total']['x'], np.array(res['P_alpha_total']['y']) / 1.0E6, label='Alpha', c='darkorange')
ax[0,0].plot(res['P_aux_total']['x'], np.array(res['P_aux_total']['y']) / 1.0E6, label='Aux', c='mediumorchid')
ax[0,0].plot(res['P_ohmic_e']['x'], np.array(res['P_ohmic_e']['y']) / 1.0E6, label='Ohmic', c='limegreen')
# ax[0,0].plot(res['P_radiation_e']['x'], np.array(res['P_radiation_e']['y']) / 1.0E6, label='P_radiation_e', c='')
ax[0,0].grid(True)
ax[0,0].legend(fontsize=5, loc='upper right')

ax[1,0].set_title('Q (Dimensionless)')
ax[1,0].plot(res['Q']['x'][5:], res['Q']['y'][5:], label='Q')
ax[1,0].set_yticks([0, 10, 20, 30, 40, 50])
ax[1,0].grid(True)

ax[2,0].set_title('Fusion Power (MW)')
ax[2,0].plot(res['P_alpha_total']['x'], 5.0 * np.array(res['P_alpha_total']['y']) / 1.0E6)
ax[2,0].grid(True)

ax[0,1].set_title('Plasma Current (MA)')
ax[0,1].plot(res['Ip']['x'], np.array(res['Ip']['y']) / 1.0E6)
ax[0,1].grid(True)

ax[2,1].set_title('Core Temperature (keV)')
ax[2,1].plot(res['T_e_core']['x'], res['T_e_core']['y'], label='T_e')
ax[2,1].plot(res['T_i_core']['x'], res['T_i_core']['y'], label='T_i')
ax[2,1].grid(True)
ax[2,1].legend()

ax[1,1].set_title('Line Average Temperature (keV)')
ax[1,1].plot(res['T_e_line_avg']['x'], res['T_e_line_avg']['y'], label='T_e')
ax[1,1].plot(res['T_i_line_avg']['x'], res['T_i_line_avg']['y'], label='T_i')
ax[1,1].grid(True)
ax[1,1].legend()

ax[0,0].set_xticks(np.linspace(0, 600, 7))
ax[0,1].set_xticks(np.linspace(0, 600, 7))
ax[1,0].set_xticks(np.linspace(0, 600, 7))
ax[1,1].set_xticks(np.linspace(0, 600, 7))
ax[2,0].set_xticks(np.linspace(0, 600, 7))
ax[2,1].set_xticks(np.linspace(0, 600, 7))

ax[0,0].set_xlabel('time (s)')
ax[1,0].set_xlabel('time (s)')
ax[2,0].set_xlabel('time (s)')
ax[0,1].set_xlabel('time (s)')
ax[1,1].set_xlabel('time (s)')
ax[2,1].set_xlabel('time (s)')

plt.tight_layout(pad=0.5)

plt.savefig('casp4.png')
plt.show()

NameError: name 'json' is not defined